### Task <br>
Let's imagine that the international cruise agency Carnival Cruise Line decided to advertise itself with the help of banners and turned to you for this. To test whether such banners are of great use, only 20 of them will be placed around the world. You need to choose 20 such locations for placement, so that the benefits are great and continue to cooperate with you. The agency is large and has several offices around the world. It wants to tie banners near these offices - it is easier to negotiate and check the result. Also, these places should be popular with tourists.

### Read data

In [15]:
import csv

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from mpl_toolkits.basemap import Basemap

In [2]:
with open('data/checkins.dat') as file:
    data = []
    for line in file:
        new_line = [x.strip() for x in line.split('|')]
        if len(new_line) == 6:
            data.append(new_line)

In [3]:
with open('data/checkins.csv', 'w') as file:
    file_writer = csv.writer(file)
    file_writer.writerows(data)

In [4]:
dataset = pd.read_csv('data/checkins.csv')
dataset.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916,5222,NaN,NaN,2012-04-21 17:39:01
1,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391,5222,NaN,NaN,2012-04-21 17:37:18
3,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840,5222,NaN,NaN,2012-04-21 17:42:58


### Data Preprocessing

In [5]:
dataset.shape

(1021966, 6)

In [6]:
%%time
dataset.dropna(inplace=True)

Wall time: 490 ms


In [7]:
dataset.shape

(396634, 6)

In [8]:
dataset.tail()

,id,user_id,venue_id,latitude,longitude,created_at
1021959,955561,626076,20073,40.850100,-73.866246,2012-04-13 09:56:48
1021960,955892,674797,2297,33.748995,-84.387982,2012-04-13 10:56:03
1021961,956377,845102,11195,42.765366,-71.467566,2012-04-13 12:08:45
1021962,956119,1139114,29488,42.439479,-83.743830,2012-04-13 11:36:44
1021964,956733,960666,60,42.331427,-83.045754,2012-04-13 21:56:19


Now we need to cluster these coordinates in order to identify the centers of tourist congestion. Since banners have a relatively small area of ​​action, we need an algorithm that allows us to limit the cluster size and does not depend on the number of clusters.

This task is a good reason to get acquainted with the MeanShift algorithm, which we bypassed in the main part of the lectures. Its description, if desired, can be viewed in the sklearn user guide, and a little later an additional video will appear with an overview of this and some other clustering algorithms. Use MeanShift, specifying bandwidth = 0.1, which in terms of degrees to meters ranges from about 5 to 10 km at mid-latitudes.

Note: on 396634 rows, clustering will take a long time. It is not forbidden to be very patient - the result will only improve. But in order to pass the task, you need a subset of the first 100 thousand lines. This is a compromise between quality and time spent. It takes about an hour to train the algorithm on the entire dataset, and about 2 minutes for 100,000 rows, but this is enough to get correct results.

Some of the resulting clusters contain too few points - such clusters are not interesting to advertisers. Therefore, it is necessary to determine which of the clusters contain, say, more than 15 elements. The centers of these clusters are optimal for placement.

### Clustering

In [9]:
import sklearn.cluster as cluster

In [10]:
X = dataset.iloc[:100000, 3:5].values
X.shape

(100000, 2)

#### MeanShift cluster

In [11]:
ms_cluster = cluster.MeanShift(bandwidth=0.1)

In [12]:
%%time
ms_cluster.fit(X)

Wall time: 3min 33s


MeanShift(bandwidth=0.1)

In [69]:
X

array([[  38.8951118,  -77.0363658],
       [  33.800745 ,  -84.41052  ],
       [  45.5234515, -122.6762071],
       ...,
       [  29.7628844,  -95.3830615],
       [  32.802955 ,  -96.769923 ],
       [  37.7749295, -122.4194155]])

In [28]:
cluster_centers = ms_cluster.cluster_centers_
len(cluster_centers)
df_centers = pd.DataFrame(cluster_centers, columns = ['lat', 'lon'])

labels = ms_cluster.labels_

In [33]:
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] += 1
        
count = 0
for key in d.keys():
    if d[key] > 15:
        count += 1
print(f"We should select {count} clusters out of {len(np.unique(labels))}")

We should select 592 clusters out of 3231


In [36]:
selected_clusters = []
for i in range(len(cluster_centers)):
    if d[i] > 15:
        selected_clusters.append(cluster_centers[i])

In [37]:
len(selected_clusters)

592

As we remember, 20 banners should be placed near the company's offices following this office addresses:<br>
33.751277, -118.188740 (Los Angeles)<br>
25.867736, -80.324116 (Miami)<br>
51.503016, -0.075479 (London)<br>
52.378894, 4.885084 (Amsterdam)<br>
39.366487, 117.036146 (Beijing)<br>
-33.868457, 151.205134 (Sydney)<br>
It remains to determine the 20 centers of clusters closest to them. Those. calculate the distance to the nearest office for each point and select the 20 with the lowest value.<br>
Note: when calculating distances and in clustering, you can neglect the fact that the Earth is round, since at points located close to each other the error is small, and at other points the value is quite large.<br>

To hand over the assignment, select from the resulting 20 centers the one that is least distant from the office nearest to it. The answer in this problem is the latitude and longitude of this center, separated by a space.<br>

In [42]:
offices = []
offices.append([33.751277, -118.188740])
offices.append([25.867736, -80.324116])
offices.append([51.503016, -0.075479])
offices.append([52.378894, 4.885084])
offices.append([39.366487, 117.036146])
offices.append([-33.868457, 151.205134])

offices[0]

[33.751277, -118.18874]

In [53]:
from scipy.spatial import distance
def euclidian_distance(office, banner):
    dst = distance.euclidean(office, banner)
    return dst

In [64]:
banner_idx = 0
office_idx = 0
min_ = euclidian_distance(office[0], selected_clusters[0])
for o_index, office in enumerate(offices):
    for index, banner in enumerate(selected_clusters):
        dst = euclidian_distance(office, selected_clusters[index])
        if min_ > dst:
            min_ = dst
            banner_idx = index
            office_idx = o_index
        

In [65]:
print(f"Min distance of banner to distance is {min_}\nThis is office {offices[office_idx]}(Sydney) and banner {selected_clusters[banner_idx]}")

Min distance of banner to distance is 0.007834758163107856
This is office [-33.868457, 151.205134](Sydney) and banner [-33.86063043 151.20477593]


In [67]:
def write_answer(center):
    with open("1.txt", "w") as f:
        f.write(str(center[0]) + ' ' + str(center[1]))

In [68]:
write_answer(selected_clusters[banner_idx])